# Getting all available Swiss petrol stations using the overpass Web API

## Libraries and settings

In [7]:
# Libraries
import os
import json
import folium
import requests
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

c:\Users\drito\Documents\SP\scientific_programming\Week_04\challenge


## Overpass turbo query to get available petrol stations

In [8]:
# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-1"="CH"][admin_level=2];
        node ["amenity"="fuel"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to .json file
with open('petrol_stations.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = pd.json_normalize(data)
df.tail() # Show last 5 rows

,type,id,lat,lon,tags.amenity,tags.fixme,tags.name,tags.operator,tags.brand,tags.fuel:diesel,...,tags.fuel:octane_87,tags.fuel:Shell_V-Power,tags.payment:post_finance,tags.covered,tags.payment:girocard,tags.payment:vpay,tags.fuel:diesel_plus,tags.construction,tags.dog,tags.payment:hubrol_card
2791,node,11660181226,47.717899,8.661321,fuel,NaN,Agrola,NaN,Agrola,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2792,node,11662707353,46.229573,7.432623,fuel,NaN,Combustia,NaN,NaN,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2793,node,11669936644,46.858436,8.639386,fuel,NaN,Hubrol,Christen Automobile AG,Hubrol,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes
2794,node,11696603774,47.030929,8.591274,fuel,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2795,node,11739615586,47.449403,7.673558,fuel,NaN,Avia,NaN,Avia,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Save data frame to a .csv file

In [9]:
# Write data frame to .csv - file
df.to_csv('petrol-stations.csv', index=False, sep=";", encoding='utf-8') #no index

## Use a pivot table to count the number of petrol stations per operator

In [10]:
# Create pivot table
tab = pd.pivot_table(df[['id', 'tags.name']],  #pivot tabel? what is that?  #df[['id', 'tags.name']] = df.loc[:, ['id', 'tags.name']] 
                       values=['id'], #what is that? #values=['id'] = id is the value that we want to count
                       index=['tags.name'], #what is that? #index=['tags.name'] = tags.name is the index that we want to count
                       aggfunc='count') #what is that? #aggfunc='count' = count the number of id's for each tags.name

# Sort pivot table
tab_sorted = tab.sort_values(by=['id'], ascending=False)
tab_sorted

,id
tags.name,
Avia,162
Agrola,148
BP,110
Tamoil,97
Shell,77
...,...
Garage du Clos,1
Garage du Gros-de-Vaud,1
Garage du Nord,1


## Plot petrol stations on map

In [11]:
# Subset of petrol stations by brand
locations = df[["lat", 
                "lon", 
                "tags.brand", 
                "tags.shop"]].loc[df["tags.brand"].isin(['Migrol', 
                                                         'Agip', 
                                                         'Shell'])]
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), #folium is a library for creating maps
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows(): 
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                   popup=location_info["tags.brand"]).add_to(map)

# Plot map
map

          lat       lon tags.brand tags.shop
8   47.686119  8.621418      Shell       NaN
19  47.495696  8.712981      Shell       NaN
23  47.381593  8.509545      Shell       NaN
28  47.381110  8.492148     Migrol       NaN
32  47.337432  8.530242      Shell       NaN


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [12]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
NT
Windows | 10
Datetime: 2024-03-20 16:17:08
Python Version: 3.11.7
-----------------------------------
